<a href="https://colab.research.google.com/github/Tessellate-Imaging/Monk_Object_Detection/blob/master/application_model_zoo/Example%20-%20Trash%20(Waste)%20Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
https://github.com/Tessellate-Imaging/Monk_Object_Detection/blob/master/application_model_zoo/Example%20-%20Nexet%20Dataset%20Vehicle%20Detection.ipynb

# Table of contents


## 1. Installation Instructions



## 2. Use trained model to detect vehicles on road



## 3. How to train using Tensorflow object detection API wrapper and nexet dataset

##  More examples on 
  - Tensorflow object detection API 1.0 - https://github.com/Tessellate-Imaging/Monk_Object_Detection/tree/master/example_notebooks/12_tf_obj_1
  - Tensorflow object detection API 2.0 - https://github.com/Tessellate-Imaging/Monk_Object_Detection/tree/master/example_notebooks/13_tf_obj_2 

# Installation

 - Run these commands
     
     - git clone https://github.com/Tessellate-Imaging/Monk_Object_Detection.git
     
     - cd Monk_Object_Detection/12_tf_obj_1/installation
     
 - Select the right file and run
 
     - chmod +x install_cuda10.sh && ./install_cuda10.sh

In [ ]:
! git clone https://github.com/Tessellate-Imaging/Monk_Object_Detection.git

In [ ]:
# For colab use the command below
! cd Monk_Object_Detection/12_tf_obj_1/installation && chmod +x install_colab.sh && ./install_colab.sh
# Restart colab runtime now

# For Local systems and cloud select the right CUDA version
# ! cd Monk_Object_Detection/12_tf_obj_1/installation && chmod +x install_cuda10.sh && ./install_cuda10.sh

In [ ]:
# Only for COLAB
# Switch to TF 1.0 version (Uncomment the following line)

#%tensorflow_version 1.x

# Now reset the runetime

In [ ]:
# Check TF version
import tensorflow as tf
print(tf.__version__)

# Use already trained model for demo

In [ ]:
import os
import sys
sys.path.append("Monk_Object_Detection/12_tf_obj_1/lib/") 

In [ ]:
from infer_detector import Infer

In [ ]:
gtf = Infer();

In [ ]:
# Download trained model

In [ ]:
! wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1bjIzyVA9b5BaMkPMcJ-IETJjbawa-W4t' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1bjIzyVA9b5BaMkPMcJ-IETJjbawa-W4t" -O obj_nexet_trained.zip && rm -rf /tmp/cookies.txt

In [ ]:
! unzip -qq obj_nexet_trained.zip

In [ ]:
gtf.set_model_params('export_dir/frozen_inference_graph.pb', "classes.txt")

In [ ]:
scores, bboxes, labels = gtf.infer_on_image("export_dir/test/1.jpg", 
                                            thresh=0.1, img_size=640, 
                                            bbox_thickness=3, text_size=2, text_thickness=4);
from IPython.display import Image
Image(filename='output.png')

In [ ]:
scores, bboxes, labels = gtf.infer_on_image("export_dir/test/2.jpg", 
                                            thresh=0.3, img_size=640, 
                                            bbox_thickness=3, text_size=2, text_thickness=4);
from IPython.display import Image
Image(filename='output.png')

In [ ]:
scores, bboxes, labels = gtf.infer_on_image("export_dir/test/3.jpg", 
                                            thresh=0.1, img_size=640, 
                                            bbox_thickness=3, text_size=2, text_thickness=4);
from IPython.display import Image
Image(filename='output.png')

In [ ]:
scores, bboxes, labels = gtf.infer_on_image("export_dir/test/4.jpg", 
                                            thresh=0.1, img_size=640, 
                                            bbox_thickness=3, text_size=2, text_thickness=4);
from IPython.display import Image
Image(filename='output.png')

In [ ]:
scores, bboxes, labels = gtf.infer_on_image("export_dir/test/5.jpg", 
                                            thresh=0.1, img_size=640, 
                                            bbox_thickness=3, text_size=2, text_thickness=4);
from IPython.display import Image
Image(filename='output.png')

# Training on the dataset

## Dataset
 - Credits: https://www.kaggle.com/solesensei/nexet-original

In [ ]:
! pip install kaggle

In [ ]:
! kaggle datasets download solesensei/nexet-original 

In [ ]:
! unzip -qq nexet-original.zip

In [ ]:
! mkdir annos

In [ ]:
import os
import sys
import numpy as np
import cv2
import pandas as pd
from pascal_voc_writer import Writer

In [ ]:
df = pd.read_csv("train_boxes.csv");

In [ ]:
df.columns

In [ ]:
from tqdm import tqdm
classes = [];
for i in tqdm(range(len(df))):
    img_name = df["image_filename"][i];
    if(not os.path.isfile("nexet/nexet/nexet_2017_1/" + img_name)):
        pass
    
    x1 = int(df['x0'][i]);
    y1 = int(df['y0'][i]);
    x2 = int(df['x1'][i]);
    y2 = int(df['y1'][i]);
    label = df["label"][i];
    if(label not in classes):
        classes.append(label)
    
    
    
    if(i == 0):
        current_img = img_name;
        img = cv2.imread("nexet/nexet/nexet_2017_1/" + img_name);
        h, w, c = img.shape;
        writer = Writer(img_name, w, h);
        writer.addObject(label, x1, y1, x2, y2);
    elif(img_name == current_img):
        writer.addObject(label, x1, y1, x2, y2);
    else:
        writer.save('annos/' + img_name.split(".")[0] + ".xml");
        
        current_img = img_name;
        img = cv2.imread("nexet/nexet/nexet_2017_1/" + img_name);
        h, w, c = img.shape;
        writer = Writer(img_name, w, h);
        writer.addObject(label, x1, y1, x2, y2);
        

        #break;
    
f = open("classes.txt", 'w')
for i in range(len(classes)):
    f.write(classes[i] + "/n");
f.close();

# Training

In [ ]:
import os
import sys
sys.path.append("Monk_Object_Detection/12_tf_obj_1/lib/")

In [ ]:
from train_detector import Detector

In [ ]:
gtf = Detector();

In [ ]:
gtf.list_models();

In [ ]:
train_img_dir = "nexet/nexet/nexet_2017_1/";
train_anno_dir = "annos";
class_list_file = "classes.txt";

gtf.set_train_dataset(train_img_dir, train_anno_dir, class_list_file, batch_size=12, trainval_split = 0.8)

In [ ]:
# Create tf record

In [ ]:
gtf.create_tfrecord(data_output_dir="data_tfrecord")

In [ ]:
# Model and hyper params

In [ ]:
gtf.set_model_params(model_name="ssd_resnet50_v1_fpn")

In [ ]:
gtf.set_hyper_params(num_train_steps=1000000, lr=0.03)

In [ ]:
gtf.export_params(output_directory="export_dir");

In [ ]:
# training
# tf.app.run() executes sys.exit() function hence cannot run in a jupyter notebook directory
# Run in a terminal - python Monk_Object_Detection/12_tf_obj_1/lib/train.py
# or
# Run the following command from notebook

In [ ]:
%run Monk_Object_Detection/12_tf_obj_1/lib/train.py

In [ ]:
# exportaing trained model
# tf.app.run() executes sys.exit() function hence cannot run in a jupyter notebook directory
# Run in a terminal - python Monk_Object_Detection/12_tf_obj_1/lib/export.py
# or
# Run the following command from notebook

In [ ]:
%run Monk_Object_Detection/12_tf_obj_1/lib/export.py

# Inference

In [ ]:
import os
import sys
sys.path.append("Monk_Object_Detection/12_tf_obj_1/lib/")

In [ ]:
from infer_detector import Infer

In [ ]:
gtf = Infer();

In [ ]:
gtf.set_model_params('export_dir/frozen_inference_graph.pb', "classes.txt")

In [ ]:
import os
img_list = os.listdir("nexet/nexet/nexet_2017_1/");

In [ ]:
scores, bboxes, labels = gtf.infer_on_image("nexet/nexet/nexet_2017_1/"+img_list[0], 
                                            thresh=0.1, img_size=640, 
                                            bbox_thickness=3, text_size=2, text_thickness=4);
from IPython.display import Image
Image(filename='output.png') 

In [ ]:
scores, bboxes, labels = gtf.infer_on_image("nexet/nexet/nexet_2017_1/"+img_list[10], 
                                            thresh=0.4, img_size=640, 
                                            bbox_thickness=3, text_size=2, text_thickness=4);
from IPython.display import Image
Image(filename='output.png') 